# Interactive Double Axis Chart for Visualization and Pattern Detection
### (in Logistic Regression Model Development)


- @author: Rajesh Jakhotia
- @company: K2 Analytics Finishing School Pvt. Ltd.
- @website: k2analytics.co.in
- @email: ar.jakhotia@k2analytics.co.in
- @blog: https://k2analytics.co.in/blog


In [1]:
##Import the packages
import pandas as pd
import os


In [2]:
os.chdir("d:/K2Analytics/Datafile") 
LR_DF = pd.read_csv("LR_DF.csv")

In [3]:
LR_DF.shape

(20000, 10)

In [4]:
LR_DF.head()

,Cust_ID,Target,Age,Gender,Balance,Occupation,No_OF_CR_TXNS,AGE_BKT,SCR,Holding_Period
0,C1,0,30,M,160378.60,SAL,2,26-30,826,9.0
1,C2,0,43,M,26275.55,PROF,23,41-45,270,23.0
2,C3,0,53,M,33616.47,SAL,45,>50,341,6.0
3,C4,0,45,M,1881.37,PROF,3,41-45,284,16.0
4,C5,0,37,M,3274.37,PROF,33,36-40,533,15.0


## Visualization and Patter Detection Code

In [5]:
def  double_axischart(var,tar,fill_na=True,bins=10):
    yaxis_label1 = "# Customers"
    yaxis_label2 = "Resp. Rate %"
    if (LR_DF[var].dtype.kind in 'biufc'):
        import numpy as np
        LR_DF['decile']=pd.qcut(LR_DF[var].rank(method='first'), bins, labels=False)
        if (fill_na== True):
                LR_DF['decile'] = LR_DF['decile'].fillna(value="Missing")
        ab=LR_DF.groupby('decile').apply(lambda x: pd.Series([
        np.min(x[var]),
        np.max(x[var]),
        np.size(x[var]),
        np.sum(x[tar]),
        ],
        index=(["min_val","max_val","total",1])
        )).reset_index()
        
        ab[var] = ab['min_val'].astype('str')+"-"+ab['max_val'].astype('str')
        res = ab[[var,"total",1]].copy()
        res.set_index(var, inplace=True)
        del LR_DF['decile']
    else:
        ab = LR_DF.groupby([var,tar]).size().reset_index()
        res = ab.pivot(index=var, columns=tar, values=0)
        res['total'] = res.sum(axis=1)
        if(fill_na==True):
            ab = LR_DF.fillna('Nan').groupby([var,tar]).size().reset_index()
            res = ab.pivot(index=var, columns=tar, values=0)
            res['total'] = res.sum(axis=1)
    res['prob'] = round(res[1]*100/res['total'],2)
    
    import matplotlib.pyplot as plt
    #plt.figure(figsize = (9,5))
    plt.rcParams['figure.figsize']=[11,6]
    ax = res[['total']].plot(kind='bar', 
                             use_index=True, label = yaxis_label1)
    plt.suptitle(var + " vs. Resp. Rate" , fontsize=20)
    plt.xlabel(var, fontsize=18)
    plt.ylabel(yaxis_label1, fontsize=16)
    L=plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    L.get_texts()[0].set_text(yaxis_label1)
    ax2 = ax.twinx()
    ax2.plot(res['prob'], linestyle='-', 
             marker='o', linewidth=2.0, 
             color="#ff8400", label = yaxis_label2)
    for i,j in res.prob.items():
        ax2.annotate(str(j)+'%', xy=(i, j))
    plt.ylabel(yaxis_label2, fontsize=16)
    L1=plt.legend(bbox_to_anchor=(1.04,0.9), loc="upper left")
    L1.get_texts()[0].set_text(yaxis_label2)

    return plt.show()


In [6]:
from ipywidgets import widgets, interactive
#Creating Widget
Variable = widgets.Dropdown(
    options=list(LR_DF.loc[:, 'Age':'Holding_Period'].columns),
    value='Age',
    description='Variable',
)


In [7]:
#Plotting
interactive(double_axischart, var=Variable, tar="Target")

interactive(children=(Dropdown(description='Variable', options=('Age', 'Gender', 'Balance', 'Occupation', 'No_…

### check out my blogs at 
https://k2analytics.co.in/blog